<a href="https://colab.research.google.com/github/Asser146/ML-Assignments/blob/master/Assignment3_Bagging_Boosting/Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from scipy.stats import pointbiserialr
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier
import pandas as pd
import numpy as np



In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install scikit-optimize
from skopt import BayesSearchCV

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 2.8 MB/s eta 0:00:00


In [11]:
data = pd.read_csv('/content/drive/MyDrive/colab_files/ML Assignment 3/output.csv')
data.head(5)

,systolic,fasting blood sugar,AST,dental caries,Gtp,hemoglobin,serum creatinine,age,height(cm),triglyceride,smoking
0,-0.353802,-1.262423,0.156700,0,0.537870,1.818767,-0.517239,-2.052535,0.536694,1.048274,1
1,0.667465,-0.479628,-0.582878,1,-0.199202,0.770702,0.597927,-0.785883,1.670617,1.138923,0
2,-0.118125,-0.479628,-0.688532,0,-0.615808,0.421347,-0.517239,-1.208100,-0.030268,-0.613639,1
3,2.160088,-0.936258,-0.899840,0,-0.647854,-1.255558,-1.074823,0.058551,-0.597229,0.383508,0
4,1.138820,0.107468,-1.005494,0,-0.647854,-0.207493,-1.074823,0.902985,-1.164191,0.564808,0


In [12]:
# Assuming df is your DataFrame and 'target' is the column name of the target variable
correlation_matrix = data.corr()
correlation_with_target = correlation_matrix['smoking'].sort_values(ascending=False)

# Display the correlation with the target variable
print(correlation_with_target)

smoking                1.000000
hemoglobin             0.441425
height(cm)             0.430535
Gtp                    0.367657
triglyceride           0.321262
serum creatinine       0.272032
dental caries          0.108755
fasting blood sugar    0.092788
systolic               0.052680
AST                    0.034115
age                   -0.193191
Name: smoking, dtype: float64


In [13]:
threshold = 0.3
# Get the features with correlation lower than the threshold
low_correlation_features = correlation_matrix[correlation_matrix['smoking'].abs() < threshold].index

# Remove the low-correlation features from the DataFrame
df_filtered = data.drop(low_correlation_features, axis=1)

# Display the updated DataFrame
df_filtered.head(5)

,Gtp,hemoglobin,height(cm),triglyceride,smoking
0,0.537870,1.818767,0.536694,1.048274,1
1,-0.199202,0.770702,1.670617,1.138923,0
2,-0.615808,0.421347,-0.030268,-0.613639,1
3,-0.647854,-1.255558,-0.597229,0.383508,0
4,-0.647854,-0.207493,-1.164191,0.564808,0


In [14]:
x = df_filtered.drop(columns = 'smoking')
y = df_filtered['smoking']
x.head(5)

,Gtp,hemoglobin,height(cm),triglyceride
0,0.537870,1.818767,0.536694,1.048274
1,-0.199202,0.770702,1.670617,1.138923
2,-0.615808,0.421347,-0.030268,-0.613639
3,-0.647854,-1.255558,-0.597229,0.383508
4,-0.647854,-0.207493,-1.164191,0.564808


In [15]:

# Split the data into training and testing sets
X_train, x_test_val, y_train, y_test_val = train_test_split(x, y, test_size=0.3, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(x_test_val, y_test_val, test_size=0.5, random_state=42)


#Bagging Model

##Hyper prameter tuning using bayesian.

In [28]:
# Define the parameter search space
param_space = {'max_depth': (1, 10),
               'min_samples_split': (2, 1000)}

# Create a BayesSearchCV object
opt = BayesSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_space,
    n_iter=10,  # Number of evaluations
    cv=3,       # Number of cross-validation folds
    n_jobs=-1,  # Number of parallel jobs (-1 means using all processors)
    random_state=42
)

# Fit the BayesSearchCV object
opt.fit(X_train, y_train)

# Get the best hyperparameters
best_params = opt.best_params_

# Create a decision tree with the best hyperparameters
best_max_depth = best_params['max_depth']
best_min_samples_split = best_params['min_samples_split']

# Use DecisionTreeClassifier with the best hyperparameters
best_tree_stump = DecisionTreeClassifier(max_depth=best_max_depth, min_samples_split=best_min_samples_split, random_state=42)
best_params_tree = opt.best_params_



##Bagging using Decision Tree

In [30]:
# Define the range of values for n_estimators in the grid search
n_estimators_values_1 = [400, 500, 600, 1000]

# Initialize variables to store the best results
best_accuracy_1 = 0
best_n_estimators_1 = 0
best_base_models_1 = None
best_predictions_list_1 = None

# Loop over different values for n_estimators
for n_estimators in n_estimators_values_1:
    # Initialize variables to store base models and predictions of base models
    base_models = []
    predictions_list = []

    # Create base models and predictions
    for i in range(n_estimators):
        # Create bootstrap sample
        indices = np.random.choice(len(X_train), size=len(X_train), replace=True)
        X_bootstrap = X_train.iloc[indices]
        y_bootstrap = y_train.iloc[indices]

        # Train base model (Decision Tree in this example)
        base_model = best_tree_stump  # Use DecisionTreeClassifier as the base model
        base_model.fit(X_bootstrap, y_bootstrap)

        # Make predictions on the validation set
        predictions = base_model.predict(X_val)

        # Store the base model and its predictions
        base_models.append(base_model)
        predictions_list.append(predictions)

    # Aggregate predictions using majority voting
    ensemble_predictions = np.mean(predictions_list, axis=0).round().astype(int)

    # Calculate accuracy
    accuracy_ensemble = accuracy_score(y_val, ensemble_predictions)

    # Update the best results if the current model is better
    if accuracy_ensemble > best_accuracy_1:
        best_accuracy_1 = accuracy_ensemble
        best_n_estimators_1 = n_estimators
        best_base_models_1 = base_models
        best_predictions_list_1 = predictions_list

# Print the best results
print("Best hyperparameters for DecisionTreeClassifier (tree stump):", best_params_tree)
print("Best number of base models for Ensemble:", best_n_estimators_1)
print("Accuracy with Ensemble:", best_accuracy_1)
best_model_1 = best_base_models_1[np.argmax(best_predictions_list_1)]

Best hyperparameters for DecisionTreeClassifier (tree stump): OrderedDict([('max_depth', 8), ('min_samples_split', 174)])
Best number of base models for Ensemble: 500
Accuracy with Ensemble: 0.7424298277950392


# **Random Forest**

In [31]:
# Specify different options for n_estimators
n_estimators_values_2 = [50, 100, 150, 200, 300, 400, 500, 600, 1000]

# Initialize variables to store the best parameters
best_accuracy_2 = 0
best_n_estimators_2 = 0
best_base_models_2 = []
best_predictions_list_2 = []

# Perform grid search
for n_estimators in n_estimators_values_2:
    # Lists to store base models and predictions of base models
    base_models = []
    predictions_list = []

    # Create bootstrap samples and train base models
    for i in range(n_estimators):
        # Create bootstrap sample
        indices = np.random.choice(len(X_train), size=len(X_train), replace=True)
        X_bootstrap = X_train.iloc[indices, :]
        y_bootstrap = y_train.iloc[indices]

        # Randomly select features for each tree
        selected_features = np.random.choice(X_train.shape[1], size=2, replace=False)
        X_bootstrap = X_bootstrap.iloc[:, selected_features]

        # Train base model (Decision Tree in this example)
        base_model = DecisionTreeClassifier(random_state=42)
        base_model.fit(X_bootstrap, y_bootstrap)

        # Make predictions on the validation set
        X_val_subset = X_val.iloc[:, selected_features]
        predictions = base_model.predict(X_val_subset)

        # Store the base model and its predictions
        base_models.append(base_model)
        predictions_list.append(predictions)

    # Aggregate predictions using majority voting
    ensemble_predictions = np.mean(predictions_list, axis=0).round().astype(int)

    # Calculate accuracy
    accuracy = accuracy_score(y_val, ensemble_predictions)

    # Update best parameters if the current model is better
    if accuracy > best_accuracy_2:
        best_accuracy_2 = accuracy
        best_n_estimators_2 = n_estimators
        best_base_models_2 = base_models
        best_predictions_list_2 = predictions_list

# Use the best parameters to print the results
print("Best n_estimators:", best_n_estimators_2)
print("Best Accuracy:", best_accuracy_2)
best_model_2 = best_base_models_2[np.argmax(best_predictions_list_2)]

Best n_estimators: 150
Best Accuracy: 0.7416925588498604


#Boosting Model

In [32]:
# Specify different options for n_estimators
n_estimators_values_3 = [5, 10, 15, 20, 25, 50]

# Initialize variables to store the best parameters
best_accuracy_3 = 0
best_n_estimators_3 = 0
best_base_models_3 = None
best_model_weights_3 = None
best_predictions_list_3 = None
best_feature_importances_3 = None

# Perform grid search
for n_estimators in n_estimators_values_3:
    # Initialize weights for the samples
    sample_weights = np.ones(len(X_train)) / len(X_train)

    # Lists to store base models, their weights, and feature importances
    base_models = []
    model_weights = []
    feature_importances_list = []

    # Perform boosting
    for _ in range(n_estimators):
        # Train a base model (tree stump)
        base_model = DecisionTreeClassifier(max_depth=1)
        base_model.fit(X_train, y_train, sample_weight=sample_weights)

        # Make predictions on the training set
        predictions = base_model.predict(X_train)

        # Calculate weighted error
        weighted_error = np.sum(sample_weights * (predictions != y_train)) / np.sum(sample_weights)

        # Calculate the base model weight
        model_weight = 0.5 * np.log((1 - weighted_error) / weighted_error)

        # Update sample weights
        sample_weights *= np.exp(-model_weight * y_train * predictions)
        sample_weights /= np.sum(sample_weights)

        # Store the base model, its weight, and feature importances
        base_models.append(base_model)
        model_weights.append(model_weight)
        feature_importances_list.append(base_model.feature_importances_)

    # Make predictions on the validation set using the weighted sum of base models
    ensemble_predictions = np.zeros_like(y_val, dtype=float)
    for i in range(n_estimators):
        predictions = base_models[i].predict(X_val)
        ensemble_predictions += model_weights[i] * predictions

    # Convert the weighted sum to binary predictions
    ensemble_predictions = np.sign(ensemble_predictions)

    # Calculate accuracy on the validation set
    accuracy = accuracy_score(y_val, ensemble_predictions)

    # Update best parameters if the current model is better
    if accuracy > best_accuracy_3:
        best_accuracy_3 = accuracy
        best_n_estimators_3 = n_estimators
        best_base_models_3 = base_models
        best_model_weights_3 = model_weights
        best_predictions_list_3 = ensemble_predictions
        best_feature_importances_3 = np.mean(feature_importances_list, axis=0)

# Use the best parameters to print the results
print("Best n_estimators:", best_n_estimators_3)
print("Best Accuracy:", best_accuracy_3)

# Display feature importances
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': best_feature_importances_3})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print("\nFeature Importances:")
print(feature_importance_df)
best_model_3 = best_base_models_3[np.argmax(best_predictions_list_3)]


Best n_estimators: 5
Best Accuracy: 0.6908736637000369

Feature Importances:
        Feature  Importance
1    hemoglobin         0.8
2    height(cm)         0.2
0           Gtp         0.0
3  triglyceride         0.0


#Final Model

In [35]:
# Final model is a ensamble of the three past models that considers the weighted vote of each of them
new_model = VotingClassifier(estimators=[
    ('model_1', best_model_1),
    ('model_2', best_model_2),
    ('model_3', best_model_3)
], voting='soft')

new_model.fit(X_train, y_train)

# Predictions on the test data
new_predictions = new_model.predict(X_test)

accuracy_new_model = accuracy_score(y_test, new_predictions)

# Print the results
print("Best Accuracy for Model 1:", best_accuracy_1)
print("Best Accuracy for Model 2:", best_accuracy_2)
print("Best Accuracy for Model 3:", best_accuracy_3)
print("Accuracy with final Model (Weighted Vote):", accuracy_new_model)

Best Accuracy for Model 1: 0.7424298277950392
Best Accuracy for Model 2: 0.7416925588498604
Best Accuracy for Model 3: 0.6908736637000369
Accuracy with final Model (Weighted Vote): 0.6865553741639897
